### Importing the Libraries

In [33]:
import scipy
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [7]:
tf.__version__ 

'2.15.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [8]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [9]:
vali_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [12]:
training_set = train_datagen.flow_from_directory('D:/SRM/Assingment/8Sem/code/2nd_git/two/dataSetHarshu/trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 24045 images belonging to 27 classes.


In [14]:
vali_set = vali_datagen.flow_from_directory('D:/SRM/Assingment/8Sem/code/2nd_git/two/dataSetHarshu/validationData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 6726 images belonging to 27 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [15]:
classifier = tf.keras.models.Sequential() 
# It creates a new object, which is a linear stack of layers. This type of model is often used for classification tasks.

#### Step 1 - Convolution

In [16]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#padding ensures that the output feature map has the same spatial dimensions as the input volume by padding zeros to the input.
#ReLU is commonly used in deep learning models for introducing non-linearity.

#### Step 2 - Pooling

In [17]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))
#pool size 2x2 window
#A stride of 2 means that the pooling window moves by 2 pixels in both the horizontal and vertical directions.
#Max pooling is a downsampling operation commonly used in convolutional neural networks (CNNs) to reduce the spatial dimensions of the input volume, 
#leading to a reduction in the number of parameters and computation in the network. It helps in controlling overfitting and improving the network's ability to learn relevant features.

#### Adding a second convolutional layer

In [18]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [19]:
classifier.add(tf.keras.layers.Flatten())
#This layer is used to flatten the input data into a one-dimensional array. 
# It converts the multi-dimensional feature maps generated by the
#convolutional layers into a one-dimensional vector,
# which can be fed into the fully connected layers.

#### Step 4 - Full Connection

In [20]:
#Each Dense layer contains units neurons. 
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

#Dropout is a regularization technique used to prevent overfitting in neural networks. It randomly sets a fraction of input units to zero during training, which helps to prevent the model from relying too heavily on any individual neurons. The parameter 0.40 specifies the fraction of units to drop during training, in this case, 40%.

### Part 3 - Training the CNN

#### Compiling the CNN

In [21]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [22]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)               4

In [23]:
pk = classifier.fit(training_set,
                  epochs = 25,
                  validation_data = vali_set)

Epoch 1/25


2405/2405 [==============================] - 829s 343ms/step - loss: 1.4829 - accuracy: 0.5389 - val_loss: 0.1128 - val_accuracy: 0.9683
Epoch 2/25
2405/2405 [==============================] - 229s 95ms/step - loss: 0.4499 - accuracy: 0.8503 - val_loss: 0.0543 - val_accuracy: 0.9823
Epoch 3/25
2405/2405 [==============================] - 232s 97ms/step - loss: 0.2936 - accuracy: 0.9069 - val_loss: 0.0272 - val_accuracy: 0.9917
Epoch 4/25
2405/2405 [==============================] - 235s 97ms/step - loss: 0.2233 - accuracy: 0.9304 - val_loss: 0.0245 - val_accuracy: 0.9926
Epoch 5/25
2405/2405 [==============================] - 239s 99ms/step - loss: 0.1866 - accuracy: 0.9407 - val_loss: 0.0164 - val_accuracy: 0.9955
Epoch 6/25
2405/2405 [==============================] - 235s 98ms/step - loss: 0.1559 - accuracy: 0.9519 - val_loss: 0.0079 - val_accuracy: 0.9976
Epoch 7/25
2405/2405 [==============================] - 241s 100ms/step - loss: 0.1398 - accuracy: 0.9581 - val_los

#### Saving the Model

In [24]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new_20epoch.h5')
print('Weights saved')

Model Saved
Weights saved


In [25]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [46]:
test_set = test_datagen.flow_from_directory('D:/SRM/Assingment/8Sem/code/2nd_git/two/dataSetHarshu/testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical',
                                            shuffle = False)

Found 3576 images belonging to 27 classes.


In [47]:
test_predict = classifier.predict(test_set)

358/358 [==============================] - 12s 32ms/step


In [54]:
STEP_SIZE_TEST=test_set.batch_size
test_set.reset()
pred=classifier.predict_generator(test_set,steps=STEP_SIZE_TEST,verbose=1)

C:\Users\harsh\AppData\Local\Temp\ipykernel_9056\1022196031.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred=classifier.predict_generator(test_set,steps=STEP_SIZE_TEST,verbose=1)


10/10 [==============================] - 0s 36ms/step


In [55]:
test_predict.shape

(3576, 27)

In [56]:
# Get the predicted labels
predicted_labels = np.argmax(test_predict, axis=-1)

# Get the true labels
true_labels = test_set.classes

print(accuracy_score(true_labels, predicted_labels))

0.9994407158836689


In [39]:
#predicted_labels.shape


(3576,)

In [40]:
#true_labels.shape

(3576,)